In [4]:
import os

print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (1845) start...
I (1845) just created a child process (3807).
I am child process (3807) and my parent is 1845.


In [5]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

Process to write: 4040
Process to read: 4041
Put A to queue...
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.


In [ ]:
#!/usr/bin/env python3
# -*- coding:utf8 -*-
from threading import Thread, Condition
import time
import random
import threading


def consumer(event, l):
    t = threading.current_thread()
    while 1:
        event_is_set = event.wait()
        if event_is_set:
            try:
                integer = l.pop()
                print('{} popped from list by {}'.format(integer, t.name))
                event.clear()  # 重置事件状态
            except IndexError:  # 为了让刚启动时容错
                pass
def producer(event, l):
    t = threading.current_thread()
    while 1:
        integer = random.choice(range(5))
        l.append(integer)
        print('{} appended to list by {}'.format(integer, t.name))
        event.set()	 # 设置事件
        time.sleep(1)
event = threading.Event()
l = []
threads = []
for name in ('consumer1', 'consumer2'):
    t = threading.Thread(name=name, target=consumer, args=(event, l))
    t.start()
    threads.append(t)
p = threading.Thread(name='producer1', target=producer, args=(event, l))
p.start()
threads.append(p)
for t in threads:
    t.join()

# # semaphore = threading.Semaphore(3)
# semaphore = threading.BoundedSemaphore(3)
#
# def fun():
#     print("Thread %s is waiting semphore\n" % threading.currentThread().getName())
#     semaphore.acquire()
#     print("Thread %s get semphore\n" % threading.currentThread().getName())
#     time.sleep(1)
#     print("Thread %s release semphore\n" % threading.currentThread().getName())
#     semaphore.release()
#
#
# if __name__ == "__main__":
#     t1 = threading.Thread(target=fun)
#     t2 = threading.Thread(target=fun)
#     t3 = threading.Thread(target=fun)
#     t4 = threading.Thread(target=fun)
#
#     t1.start()
#     t2.start()
#     t3.start()
#     t4.start()
#
#     t1.join()
#     t2.join()
#     t3.join()
#     t4.join()
#
#     semaphore.release()  #这里因为是简单的Semaphore,所以可以再次释放，不会报错，而BoundedSemaphore，则会报错


# queue = []
# MAX_NUM = 10
# condition = Condition()
#
# class ProducerThread(Thread):
#     def run(self):
#         nums = range(5)
#         global queue
#         while True:
#             condition.acquire()
#             if len(queue) == MAX_NUM:
#                 print('queue is full')
#                 condition.wait()
#                 print('got notified, space in queue')
#             num = random.choice(nums)
#             queue.append(num)
#             print('produced %s' % num)
#             condition.notify()
#             condition.release()
#             time.sleep(random.random())
#
# class ConsumerThread(Thread):
#     def run(self):
#         global queue
#         while True:
#             condition.acquire()
#             if not queue:
#                 print("Nothing in queue, consumer is waiting")
#                 condition.wait()
#                 print("Producer added something to queue and notified the consumer")
#             num = queue.pop(0)
#             print('consumed %s' % num)
#             condition.notify()
#             condition.release()
#             time.sleep(random.random())
#
#
# ProducerThread().start()
# ConsumerThread().start()


# from threading import Thread
# import time
# import random
# from Queue import Queue
#
# queue = Queue(10)
#
# class ProducerThread(Thread):
#     def run(self):
#         nums = range(5)
#         global queue
#         while True:
#             num = random.choice(nums)
#             queue.put(num)
#             print "Produced", num
#             time.sleep(random.random())
#
# class ConsumerThread(Thread):
#     def run(self):
#         global queue
#         while True:
#             num = queue.get()
#             queue.task_done()
#             print "Consumed", num
#             time.sleep(random.random())
#
# ProducerThread().start()
# ConsumerThread().start()
